In Linux, the environment variable `LD_LIBRARY_PATH` is a colon-separated set of directories where libraries should be searched for first, before the standard set of directories; this is useful when debugging a new library or using a nonstandard library for special purposes. - [source](https://askubuntu.com/questions/844578/what-does-manpath-ld-library-path-mean-in-linux)

__NOTE__: Run it before launching `jupyter`

In [1]:
!export LD_LIBRARY_PATH=/usr/local/cuda-10.0/lib64

In [2]:
import tensorflow as tf

tf.test.is_built_with_cuda()
tf.test.is_gpu_available(cuda_only=False, min_cuda_compute_capability=None)

True

Clear the GPU if needed

In [3]:
!nvidia-smi

Tue Mar 30 15:31:27 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.87.01    Driver Version: 418.87.01    CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla T4            On   | 00000000:00:1E.0 Off |                    0 |
| N/A   31C    P0    26W /  70W |    111MiB / 15079MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [15]:
from numba import cuda 

device = cuda.get_current_device()
device.reset()

In [3]:
!nvidia-smi

Wed Feb 17 05:57:35 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.87.01    Driver Version: 418.87.01    CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla T4            On   | 00000000:00:1E.0 Off |                    0 |
| N/A   30C    P0    26W /  70W |    111MiB / 15079MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [3]:
from deeppavlov.dataset_readers.basic_classification_reader import BasicClassificationDatasetReader
from deeppavlov.dataset_iterators.basic_classification_iterator import BasicClassificationDatasetIterator

from deeppavlov.core.data.simple_vocab import SimpleVocabulary

from deeppavlov.models.preprocessors.bert_preprocessor import BertPreprocessor
from deeppavlov.models.preprocessors.one_hotter import OneHotter
from deeppavlov.models.bert.bert_classifier import BertClassifierModel

import pandas as pd
import numpy as np

import datetime
import time
import re
import ast

from tqdm import tqdm_notebook

import os

import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_colwidth', -1)

[nltk_data] Downloading package punkt to
[nltk_data]     /home/a-dbaiturs@PMICLOUD.GLOBAL/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/a-dbaiturs@PMICLOUD.GLOBAL/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package perluniprops to
[nltk_data]     /home/a-dbaiturs@PMICLOUD.GLOBAL/nltk_data...
[nltk_data]   Package perluniprops is already up-to-date!
[nltk_data] Downloading package nonbreaking_prefixes to
[nltk_data]     /home/a-dbaiturs@PMICLOUD.GLOBAL/nltk_data...
[nltk_data]   Package nonbreaking_prefixes is already up-to-date!


In [4]:
PATH = '.'
PATH_RUBERT = PATH + r'/ru_bert_tf'
PATH_RESULTS = PATH + r'/modelResults'
PATH_COMMENT_SOURCE = PATH + r'/prodData/source'
PATH_COMMENT_RESULTS = PATH + r'/prodData/results'
PATH_TOPICS = PATH + r'/topics/TopicsMap.xlsx'

# Load model

In [5]:
!ls $PATH_RESULTS

model_020321  model_020321Topics


In [6]:
modelLoad = r'/model_020321Topics'

PATH_LOAD = PATH_RESULTS + modelLoad
classes = np.load(PATH_LOAD + r'/classes.npy', allow_pickle=True)

bert_preprocessor = BertPreprocessor(
    vocab_file=PATH_LOAD + r'/vocab.txt',
    do_lower_case=False,
    max_seq_length=64
)

bert_classifier = BertClassifierModel(
    n_classes = len(classes),
    return_probas = True,
    one_hot_labels = True,
    bert_config_file = PATH_RUBERT + r'/bert_config.json',
    pretrained_bert = r'bert_model.ckpt',
    save_path = PATH_LOAD + r'/model',
    load_path = PATH_LOAD + r'/model',
    keep_prob = 0.5,
    learning_rate = 1e-05,
    min_learning_rate = 1e-07,
    learning_rate_drop_patience = 5,
    learning_rate_drop_div = 2.0,
    multilabel = True
)

bert_classifier.load()








The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Please use `layer.__call__` method instead.

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where




Instructions for updating:
Use standard file APIs to check for files with this prefix.


2021-04-20 14:15:26.460 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 51: [loading model from /home/a-dbaiturs@PMICLOUD.GLOBAL/Projects/Project1/bertNewModel/modelResults/model_020321Topics/model]



INFO:tensorflow:Restoring parameters from /home/a-dbaiturs@PMICLOUD.GLOBAL/Projects/Project1/bertNewModel/modelResults/model_020321Topics/model


2021-04-20 14:15:27.976 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 51: [loading model from /home/a-dbaiturs@PMICLOUD.GLOBAL/Projects/Project1/bertNewModel/modelResults/model_020321Topics/model]


INFO:tensorflow:Restoring parameters from /home/a-dbaiturs@PMICLOUD.GLOBAL/Projects/Project1/bertNewModel/modelResults/model_020321Topics/model


Load thresholds

In [7]:
ts = np.load(PATH_LOAD + r'/ts.npy')

Load data

In [8]:
!ls prodData/source

exportNew.csv


In [9]:
df = pd.read_csv(PATH_COMMENT_SOURCE + r'/exportNew.csv', encoding='windows-1251')
print(df.shape)
df.sample(5)

(36600, 3)


,ID,Comment,Comment_EN
29821,R_2rkXCAc2WEkWe6r,NaN,NaN
32751,R_2RQWHJ6jyMUTlSA,Мне даже не позвонили!!!!!!,I didn't even get a call!!!!!!
9982,R_2agfwyGpi2MbX7k,NaN,NaN
10208,R_2fHgFwyH8JCXed6,NaN,NaN
13565,R_1qhekYWgMl4B7Wf,NaN,NaN


In [10]:
df_comment = df[df.Comment.notna()]
df_nocomment = df[df.Comment.isna()]
print(df.ID.nunique(), df_comment.ID.nunique(), df_nocomment.ID.nunique())

36600 19241 17359


In [11]:
def get_preds(comments, verbose = False):
    y_preds = []
    
    if verbose:
        print(f'Predicting comments')
        for i in tqdm_notebook(range(0, len(comments), 256)):
            x = comments.iloc[i:i + 256]

            y_pred = bert_classifier(bert_preprocessor(x))
            y_preds.append(y_pred)
    else:
        for i in range(0, len(comments), 256):
            x = comments.iloc[i:i + 256]

            y_pred = bert_classifier(bert_preprocessor(x))
            y_preds.append(y_pred)
    y_preds = np.concatenate(y_preds, axis=0)
    
    return y_preds

def clean_preds(preds, ts):
    ind_unc = list(classes).index('Uncategorized')
    
    preds_bin = (preds > ts).astype(int)
    preds_bin[(preds_bin[:, ind_unc] == 1) & 
              (preds_bin.sum(axis=1) > 1), ind_unc] = 0
    preds_bin[preds_bin.sum(axis=1) == 0, 0] = 1
    
    return preds_bin

def merge_preds(df_comment, df_nocomment, preds):
    res = pd.DataFrame(data = preds, columns = classes)
    res = pd.concat([df_comment.reset_index(drop=True), res], 
                    axis = 1, 
                    ignore_index = False)
    res = res.melt(id_vars = ['ID', 'Comment', 'Comment_EN'], 
                   var_name = 'l3', value_name = 'Value')
    res = res[res['Value'] == 1][['ID', 'Comment', 'Comment_EN', 'l3']]
    df_nocomment['l3'] = 'No comment'
    res = pd.concat([df_nocomment, res]).reset_index(drop = True)
    return res

def print_samples(df, sampleSize = 5):
    for c in classes:
        print(c)
        if df[df['l3'] == c].shape[0] > sampleSize:
            for i, comment in enumerate(df[df['l3'] == c].Comment.sample(sampleSize).values):
                print(f'{i+1}) {comment}')
        print('\n\n')

preds = get_preds(df_comment.Comment, verbose = True)
preds = clean_preds(preds, ts)
res = merge_preds(df_comment, df_nocomment, preds)
print_samples(res)

Predicting comments



Alternative to cigarette
1) Кажется, что это лучшая альтернатива курению сигарет
2) Потому что это однозначно лучше обычных сигарет
3) По крайней мере , думаю что меньше вреда здоровью 
4) Мне очень понравился,  я раньше сигарету курил много как купил IQOS бросил супер просто без слов от IQOS мало вреда для здоровья чем сигарета 
5) Курить стала меньше, и нет запаха табака как от обычных сигарет.



Ash
1) Ни дыма, но пепла, ни запаха изо рта и от вещей. Супер инновационная вещь. Всем буду рекомендовать. 
2) Нет запаха горелого, нет мусора
3) не имеет запаха, легко использовать, не оставляет после себя пепел.
4) Нет запаха, пепла! 
5) Отсутствие пепла и запаха табака, меньшее количество затяжек по сравнению с обычными сигаретами



Battery
1) Здравствуйте, купила недавно вашу продукцию, первые дни было все нормально, но потом стики плохо куриться стали, очень слабо. В чем может быть причина? Очень не приятная ситуация, пользуюсь очень аккуратно, чищу вовремя
2) Заряда хватает на 1,5 с

In [12]:
print(res.ID.nunique(), res[res['l3'] != 'No comment'].ID.nunique(), res[res['l3'] == 'No comment'].ID.nunique())

36600 19241 17359


In [13]:
topics = pd.read_excel(PATH_TOPICS, engine='openpyxl')
topics.sample(5)

,l1,l2,l3
18,L1_2: Operations & Processes,"L2_2: Order, delivery & returns",L3_2: Checkout and payment process
14,L1_1: Brand & Marketing,L2_1: Marketing campaigns,L3_1: Marketing frequency
3,L1_1: Brand & Marketing,L2_1: Brand positioning,L3_1: Other
31,L1_3: Product,L2_3: Device & accessories,L3_3: Ease of use
9,L1_1: Brand & Marketing,L2_1: Channel specifics,L3_1: Store size and layout


In [14]:
def merge_topics(res, topics):
    topics = topics.applymap(lambda x: x[6:])
    res = pd.merge(res, topics, how='left', on='l3')
    return res[['ID', 'Comment', 'Comment_EN', 'l1', 'l2', 'l3']]
res = merge_topics(res, topics)
res.sample(10)

,ID,Comment,Comment_EN,l1,l2,l3
1608,R_1gdoPOLJohYY7cs,NaN,NaN,No comment,No comment,No comment
17216,R_yKmSI5jcdf1IAIF,NaN,NaN,No comment,No comment,No comment
19300,R_3PcKKmdIQDvLJ7R,"Стильный удобный ,минусов нет","Stylish handy, no minuses",Brand & Marketing,Brand positioning,Alternative to cigarette
11991,R_3PAl2el9ztAp0NQ,NaN,NaN,No comment,No comment,No comment
31146,R_UbFMiIKLEQrq1r3,"Грамотный консультант, все отлично объяснила.","A competent consultant, she explained everything perfectly.",Sales & Customer Service,Service quality,Staff reliability
8203,R_3eay32xvc28yhwI,NaN,NaN,No comment,No comment,No comment
12276,R_3w36BItHCuHWr3r,NaN,NaN,No comment,No comment,No comment
22925,R_eFlVXxtzUH0kJpL,Понравилось всё. Жаль что нет скидки постоянным клиентам,I liked everything. Too bad there is no discount to regular customers,Sales & Customer Service,Price,Discounts/ promotion
12358,R_56c3sdv8STfDVpD,NaN,NaN,No comment,No comment,No comment
7380,R_2Y8HmgLqE0PQJ0N,NaN,NaN,No comment,No comment,No comment


In [15]:
res.ID.nunique()

36600

In [16]:
res.l3.value_counts()

No comment                                     17359
Uncategorized                                  8129 
Alternative to cigarette                       3838 
Staff reliability                              2932 
Smoke & smell                                  2134 
Order & delivery details accuracy              1094 
Friendliness                                   961  
Ease of use                                    758  
Expertise                                      468  
Charger, holder or accessories reliability     423  
Taste                                          387  
Sticks reliability                             365  
Design/ visual appeal                          318  
Return/ replacement process                    263  
Can use everywhere                             259  
Price                                          253  
Discounts/ promotion                           251  
Social inclusion                               220  
Ash                                           

In [17]:
ind = 0
print(classes[ind])
res[res['l3'] == classes[ind]]['Comment'].sample(10)

Alternative to cigarette


20218    Потому что полностью отказвлся от сигарет, нравиться что нет запаха и не нужно выходить на уличу что бы покурить \n
21033    Нет запаха,Понравилось что нет запаха, не нужна зажигалка или спички                                               
19994    Мне не понравились, я начала кашлять                                                                               
18943    Намного легче чем сигареты, приятнее курить , накуривает хорошо                                                    
18165    Я рад что попробывал эйкос и бросил сигареты,Не ожидал что помогут решить проблему. Очень рад спасибо              
21057    очень приятное обслуживание, наглядная демонстрация разницы между электронными сигаретами и обычными               
19771    IQos сам по себе хорош\nНет зависимости \n                                                                         
18972    IQOS лучше чем сигареты, последняя версия очень нравится ??????                                                    


In [18]:
def save_results(res):
    res = res[['ID', 'l1', 'l2', 'l3']]
    res.to_excel(PATH_COMMENT_RESULTS + r'/results.xlsx', encoding='windows-1251', index=False)
save_results(res)